# TODO:
#### Automatic data retrieval from google forms for during-suvrvey checking
[X] spreadsheets url inference <br>
[X] gather results in python script

#### Qualification filtering
[X] ~50 qualifications

#### Frauder detection
[X] > X ratio repeated ===> discarded

#### Honeypots
[X] place honeypots from a defined set in each form<br>
[X] honeypot frauders detection

#### MTurk templating
[X] Create guidelines for the workers

#### Gform templating
[X] Remove index of the emojis and add separator btwn each question NB: not practical

#### Emojis png
[X] Rerun creation of emojis png + upload on drive

___

#### MTurk features
[X] Implement dry runs <br>
[ ] Link between MTurk framework and gdrive service (not same ids etc) ==> visualization of the actual results for a given workerid + hitid (not just the completion code as before)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import create_mturk_client,clean_own_worker,Turker,MTurkparam
from src.data.watcher import Watcher
from src.visualization.turk_widgets import get_buttons
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results,download_drive_spreadsheet
from src.data.fraudulous import detect_repeat_frauders,detect_honey_frauders
from src.utils import generate_password 
from src.constants import WATCHER_PATH,NMB_FORMS_THRESHOLD,TOKEN_PATH,CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH, FORMS_RESULTS_DIR,WATCHER_FORMS_RESULTS_DIR,HONEYPOTS
from src.utils import read_access_keys
import subprocess
import pandas as pd
import jdc
from IPython.display import clear_output
import pickle as pk
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

import ipywidgets as widgets
from src.utils import get_form_urls


## Urls info retrieval

In [4]:
############ TODO: copy-paste the id of the most recent version of the url file ############
file_id = "1vYOWZ6jHJWlnnbFH5pEA5HMQ64aHepZe5swvoCETigE"
############################################################################################
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
formidx2url,formidx2gid = get_form_urls(service,file_id)
download_all_csv_results(formidx2gid,FORMS_RESULTS_DIR,service)

## MTurk management

[ ] Monitor > 2 <br>
[ ] Honeypots <br>
[ ] Repeat <br>
[ ] Fake id <br>
[ ] Dry runs

In [21]:
param = MTurkparam(hitlayout="3VQCRCGMCT2NU7RBSKK6J2PIFKVQ77",
             MaxAssignments = 10,
             LifetimeInSeconds = 600,
             AutoApprovalDelayInSeconds=600,
             AssignmentDurationInSeconds=600,
             Reward='0.01',
             n_forms=len(formidx2gid))

Estimated cost:0.40 $


In [6]:
turk = Turker(param=param,
              gservice=service,
              formidx2url=formidx2url,
              formidx2gid=formidx2gid,
              formrespath=FORMS_RESULTS_DIR)

Loading hit2form


In [9]:
turk.watcher_process

In [10]:
get_buttons(turk,file_id)

Button(button_style='primary', description='stop monitor', style=ButtonStyle())

In [10]:
clean_own_worker(turk.client)

Worker already clean


https://workersandbox.mturk.com/mturk/

## Qualifications

In [ ]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=0
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

In [ ]:
turker.client.update_qualification_type(
    QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN',
    QualificationTypeStatus='Active',
    Test='string',
    AnswerKey='string',
    TestDurationInSeconds=123,
    RetryDelayInSeconds=123,
    AutoGranted=False,
    AutoGrantedValue=123
)